In [1]:
#Question 1
#Access the viswiz dataset

import os
import json
import requests
import pandas as pd

base_url = 'https://ivc.ischool.utexas.edu/VizWiz/data'
split_train = 'train'
split_test = 'test'
split_val = 'val'
trainFile = '%s/Annotations/%s.json'%(base_url, split_train)
testFile = '%s/Annotations/%s.json'%(base_url, split_test)
valFile = '%s/Annotations/%s.json'%(base_url, split_val)


annotations_train = requests.get(trainFile, allow_redirects=True)
annotations_val = requests.get(valFile, allow_redirects=True)
annotations_test = requests.get(testFile, allow_redirects=True)

In [2]:
#get raw train, validation and test data
labels_train = annotations_train.json()
labels_test = annotations_test.json()
labels_val = annotations_val.json()

image_name_train = []
question_train = []
label_train = []

image_name_test = []
question_test = []
label_test = []

image_name_val = []
question_val = []
label_val = []

for vq in labels_train[0:5000]:
    image_name_train.append(vq['image'])
    question_train.append(vq['question'])
    label_train.append(vq['answerable'])
    
for vq in labels_val[0:1000]:
    image_name_val.append(vq['image'])
    question_val.append(vq['question'])
    label_val.append(vq['answerable'])
    
for vq in labels_test[0:100]:
    image_name_test.append(vq['image'])
    question_test.append(vq['question'])
    #label_test.append(vq['answerable'])

image_name_train = pd.DataFrame(image_name_train,columns=['image'])
question_train = pd.DataFrame(question_train,columns=['question'])
X_train_raw = pd.concat([image_name_train,question_train],axis=1)
y_train_raw = pd.DataFrame(label_train,columns=['label'])

image_name_test = pd.DataFrame(image_name_test,columns=['image'])
question_test = pd.DataFrame(question_test,columns=['question'])
X_test_raw = pd.concat([image_name_test,question_test],axis=1)
y_test_raw = pd.DataFrame(label_test,columns=['label'])

image_name_val = pd.DataFrame(image_name_val,columns=['image'])
question_val = pd.DataFrame(question_val,columns=['question'])
X_val_raw = pd.concat([image_name_val,question_val],axis=1)
y_val_raw = pd.DataFrame(label_val,columns=['label'])

In [37]:
y_train = y_train_raw.loc[0:699].append(y_train_raw[800:3100]).append(y_train_raw[3200:3600]).append(y_train_raw[3800:4400])
y_train.to_csv('y_train.csv',header=False)
y_val = y_val_raw.loc[0:199].append(y_val_raw[300:900])
y_val.to_csv('y_val.csv',header=False)


(4000, 1)

In [3]:
# Use Microsoft's API to extract key tags from images
subscription_key = ''
vision_base_url = 'https://westus.api.cognitive.microsoft.com/vision/v1.0'
vision_analyze_url = vision_base_url + '/analyze?'
def extract_features(data):
    return {
        "tags": data["description"]['tags']  
    }
def analyze_image(image_url):
    # Microsoft API headers, params, etc
    headers = {'Ocp-Apim-Subscription-key': subscription_key}
    #params = {'visualfeatures': 'Adult,Categories,Description,Color,Faces,ImageType,Tags'}
    params = {'visualfeatures': 'Description'}
    data = {'url': image_url}
    
    # send request, get API response
    response = requests.post(vision_analyze_url, headers=headers, params=params, json=data)
    response.raise_for_status()
    analysis = response.json()
    return analysis

def get_image_tags(X_raw,n):
    image_tags = []
    for i in range(100):
        image_url = 'https://ivc.ischool.utexas.edu/VizWiz/data/Images/%s'%(X_raw['image'][i+n])
        data = extract_features(analyze_image(image_url))
        tag_i=[]
        for item in data['tags']:
            tag_i.append(item)
        tag_i_join = ' '.join(tag_i)
        #features = pd.DataFrame(extract_features(data))
        image_tags.append(tag_i_join)
    return image_tags

In [4]:
#Use Microsoft's API to extract key phrases from the questions
subscription_key_text = ''
text_analytics_base_url = 'https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/'
key_phrase_api_url = text_analytics_base_url + 'keyPhrases'

def get_questions(question_raw,n):
    questions = []
    for i in range(100):
        question_json = (question_raw['question'][i+n])
        documents = {'documents': [{'id': i, 'text': question_json}]}
        headers = {"Ocp-Apim-Subscription-Key": subscription_key_text}
        response = requests.post(key_phrase_api_url,headers = headers,json=documents)
        #questions.append(response.json()['documents'])
        question_json = response.json()['documents']
        question = pd.DataFrame(question_json)['keyPhrases']
        question = question.tolist()[0]
        tag_i=[]
        for item in question:
            tag_i.append(item)
        question = ' '.join(tag_i)
        questions.append(question)
    
    return questions

In [35]:
%%time
#Write text image tags to .csv file
import csv
#Get training/valid/test data of image tags
image_tags = []
for i in range(42,44):
    image_tags = get_image_tags(X_train_raw,100*i)
    data = pd.DataFrame(data=image_tags)
    data.to_csv('image_tags_train.csv',mode='a',header=False)

#train (38,40)now skip 7-8 31-32 36-38 len = range(n-2)
#val  skip 2-3

Wall time: 5min 11s


In [36]:
%%time
#Write text question tags to .csv file
#Get training/valid/test data of questions
text_questions = []
for i in range(42,44):
    text_questions = get_questions(question_train,100*i)
    data = pd.DataFrame(data=text_questions)
    data.to_csv('questions_train.csv',mode='a',header=False)
    #38,40

Wall time: 6 s
